In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
#!git clone 'https://github.com/ThanhChinhBK/vietnews'

Cloning into 'vietnews'...
remote: Enumerating objects: 143827, done.
remote: Counting objects: 100% (143827/143827), done.
remote: Compressing objects: 100% (143815/143815), done.
remote: Total 143827 (delta 11), reused 143827 (delta 11), pack-reused 0
Receiving objects: 100% (143827/143827), 194.68 MiB | 17.15 MiB/s, done.
Resolving deltas: 100% (11/11), done.
Checking out files: 100% (150704/150704), done.


In [ ]:
article_list = []

for file in os.listdir('/content/vietnews/data/train_tokenized'):
  #print(file)
  with open('/content/vietnews/data/train_tokenized/' + file, 'r') as f:
    article = f.read().splitlines()
    article_list.append(article[4:-1])   # leave out titles, abstracts, image subtitles
    f.close()

In [ ]:
article_list[0]

['Cảnh_sát Canada vừa công_bố nguyên_nhân cái chết của vợ_chồng tỉ_phú Barry_Sherman và bà Honey , song kết_luận sơ_bộ của vụ_việc chỉ góp thêm bí_ẩn cho vụ_việc chấn_động này .',
 'Nhà sáng_lập hãng dược Apotex , ông Barry_Sherman , 75 tuổi , và vợ , bà Honey_Sherman , 70 tuổi , được tìm thấy đã chết tại nhà_riêng ở North_York hôm 15-12 .',
 'Cảnh_sát Toronto đánh_giá vụ_việc là " đáng ngờ " nên đã yêu_cầu lực_lượng điều_tra án_mạng vào_cuộc , theo thông_tin từ Đài CBC_News ( Canada ) ngày 18-12 .',
 'Trong thông_cáo phát đi hôm 17-12 ( giờ_địa_phương ) , cảnh_sát Toronto tuyên_bố cả hai nạn_nhân đều bị " ngạt thở do dây buộc " , nghĩa_là bị xiết cổ , cho đến chết .',
 'Tuy_nhiên , điều khó hiểu là các điều_tra_viên tuyên_bố họ sẽ không tìm_kiếm thêm nghi_can nào khác trong vụ_việc , theo Sky_News .',
 'Một nguồn_tin cảnh_sát trước đó cho CBC News biết một người ngoài gia_đình Sherman đã phát_hiện ra thi_thể vợ_chồng tỉ_phú , song cảnh_sát không tiết_lộ gì nhiều về quá_trình điều_tra 

In [ ]:
for i in range(len(article_list)):
  article_list[i] = list(filter(None, article_list[i]))
  article_list[i] = ' '.join(article_list[i])
  #article_list[i] = article_list[i].replace('_', ' ')

In [ ]:
article_list[0]

'Cảnh_sát Canada vừa công_bố nguyên_nhân cái chết của vợ_chồng tỉ_phú Barry_Sherman và bà Honey , song kết_luận sơ_bộ của vụ_việc chỉ góp thêm bí_ẩn cho vụ_việc chấn_động này . Nhà sáng_lập hãng dược Apotex , ông Barry_Sherman , 75 tuổi , và vợ , bà Honey_Sherman , 70 tuổi , được tìm thấy đã chết tại nhà_riêng ở North_York hôm 15-12 . Cảnh_sát Toronto đánh_giá vụ_việc là " đáng ngờ " nên đã yêu_cầu lực_lượng điều_tra án_mạng vào_cuộc , theo thông_tin từ Đài CBC_News ( Canada ) ngày 18-12 . Trong thông_cáo phát đi hôm 17-12 ( giờ_địa_phương ) , cảnh_sát Toronto tuyên_bố cả hai nạn_nhân đều bị " ngạt thở do dây buộc " , nghĩa_là bị xiết cổ , cho đến chết . Tuy_nhiên , điều khó hiểu là các điều_tra_viên tuyên_bố họ sẽ không tìm_kiếm thêm nghi_can nào khác trong vụ_việc , theo Sky_News . Một nguồn_tin cảnh_sát trước đó cho CBC News biết một người ngoài gia_đình Sherman đã phát_hiện ra thi_thể vợ_chồng tỉ_phú , song cảnh_sát không tiết_lộ gì nhiều về quá_trình điều_tra . Tờ The_Globe and Ma

In [165]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(article_list)

In [166]:
print(tfidf_matrix.shape)

(105418, 238217)


Kết thúc preprocessing

In [167]:
tfidf_matrix[0,tfidf.vocabulary_['nhật']]

0.0

In [199]:
def score_sentence(sentence, a_index_):
  sentence_score = 0
  sentence = sentence.replace(' ,', '').strip()
  for i_word in sentence.split(' '):
    #print(i_word)
    sentence_score += tfidf_matrix[a_index_, tfidf.vocabulary_.get(i_word.lower(), 217377)]
  
  return sentence_score


def score_article(article, a_index_):
  sentence_score_list = []
  sentence_list = article.split('.')

  for i_sentence in sentence_list:
    sentence_score = score_sentence(i_sentence, a_index_)
    sentence_score_list.append(sentence_score)

  return sentence_score_list, sentence_list


def generate_summary(sentence_score_array, sentence_list):
  sentence_score_array = np.asarray(sentence_score_list)
  mean_score = np.mean(sentence_score_array)
  sentence_list_arr = np.asarray(sentence_list, dtype=object)
  summary = sentence_list_arr[sentence_score_array > 1.15*mean_score]
  summary = ' '.join(summary)

  return summary


In [187]:
a_index_ = 0
summary_list = []

for i_article in article_list[:50]:
  #print(a_index_)
  #print(i_article)
  sentence_score_list, sentence_list = score_article(i_article, a_index_)
  summary = generate_summary(sentence_score_array, sentence_list)
  summary_list.append(summary)
  #a_index_ += 1

In [168]:
tfidf_matrix[0, tfidf.vocabulary_['và']]

0.05784704989977959

In [153]:
np.argmin(np.sum(tfidf_matrix, axis=0))

217377

In [172]:
np.sum(tfidf_matrix[:, 30], axis=0)

matrix([[0.25974146]])

In [188]:
summary_list

['Cảnh_sát Canada vừa công_bố nguyên_nhân cái chết của vợ_chồng tỉ_phú Barry_Sherman và bà Honey , song kết_luận sơ_bộ của vụ_việc chỉ góp thêm bí_ẩn cho vụ_việc chấn_động này   Cảnh_sát Toronto đánh_giá vụ_việc là " đáng ngờ " nên đã yêu_cầu lực_lượng điều_tra án_mạng vào_cuộc , theo thông_tin từ Đài CBC_News ( Canada ) ngày 18-12   Một nguồn_tin cảnh_sát trước đó cho CBC News biết một người ngoài gia_đình Sherman đã phát_hiện ra thi_thể vợ_chồng tỉ_phú , song cảnh_sát không tiết_lộ gì nhiều về quá_trình điều_tra   Tờ The_Globe and Mail dẫn một nguồn thạo tin cho biết người phát_hiện thi_thể là một nhân_viên môi_giới địa_ốc , vốn đang thu_xếp bán ngôi nhà của ông_bà Sherman   Bất_chấp sự " kín_tiếng " của cảnh_sát , nhiều hãng tin đã dẫn chính nguồn_tin từ cảnh_sát cho_hay vụ_việc đang được điều_tra theo hướng " giết người rồi tự_tử " , tức một trong hai người đã đoạt mạng người kia trước khi tự kết_liễu bản_thân   Tuy_nhiên , trong thông_cáo phát đi ngày 16-12 , gia_đình Sherman khẳn

In [189]:
text = 'Cảnh_sát Canada vừa công_bố nguyên_nhân cái chết của vợ_chồng tỉ_phú Barry_Sherman và bà Honey , song kết_luận sơ_bộ của vụ_việc chỉ góp thêm bí_ẩn cho vụ_việc chấn_động này . Nhà sáng_lập hãng dược Apotex , ông Barry_Sherman , 75 tuổi , và vợ , bà Honey_Sherman , 70 tuổi , được tìm thấy đã chết tại nhà_riêng ở North_York hôm 15-12 . Cảnh_sát Toronto đánh_giá vụ_việc là " đáng ngờ " nên đã yêu_cầu lực_lượng điều_tra án_mạng vào_cuộc , theo thông_tin từ Đài CBC_News ( Canada ) ngày 18-12 . Trong thông_cáo phát đi hôm 17-12 ( giờ_địa_phương ) , cảnh_sát Toronto tuyên_bố cả hai nạn_nhân đều bị " ngạt thở do dây buộc " , nghĩa_là bị xiết cổ , cho đến chết . Tuy_nhiên , điều khó hiểu là các điều_tra_viên tuyên_bố họ sẽ không tìm_kiếm thêm nghi_can nào khác trong vụ_việc , theo Sky_News . Một nguồn_tin cảnh_sát trước đó cho CBC News biết một người ngoài gia_đình Sherman đã phát_hiện ra thi_thể vợ_chồng tỉ_phú , song cảnh_sát không tiết_lộ gì nhiều về quá_trình điều_tra . Tờ The_Globe and Mail dẫn một nguồn thạo tin cho biết người phát_hiện thi_thể là một nhân_viên môi_giới địa_ốc , vốn đang thu_xếp bán ngôi nhà của ông_bà Sherman . Báo Globe and Mail khẳng_định không có dấu_hiệu ngôi nhà đã bị xâm_nhập và không có lời nhắn hay thư tuyệt_mệnh gì để lại hiện_trường . Bất_chấp sự " kín_tiếng " của cảnh_sát , nhiều hãng tin đã dẫn chính nguồn_tin từ cảnh_sát cho_hay vụ_việc đang được điều_tra theo hướng " giết người rồi tự_tử " , tức một trong hai người đã đoạt mạng người kia trước khi tự kết_liễu bản_thân . Tuy_nhiên , trong thông_cáo phát đi ngày 16-12 , gia_đình Sherman khẳng_định " cha_mẹ chúng_tôi đã sống rất_đỗi nhiệt_thành và luôn hướng về gia_đình và cộng_đồng , điều này hoàn_toàn trái_ngược với các tin_đồn đáng tiếc trên mạng xã_hội xoay quanh cái chết của họ " . Gia_đình Sherman cũng chỉ_trích các nguồn_tin cảnh_sát là " vô_trách_nhiệm " khi thông_tin không đúng cho truyền_thông . " ( Giả_thuyết đây là vụ giết người rồi tự_tử ) là thứ mà không ai trong gia_đình , bạn_bè hay đồng_nghiệp ( của cha_mẹ chúng_tôi ) tin là thật " - thông_cáo viết . Gia_đình nạn_nhân vì_thế yêu_cầu cảnh_sát tiến_hành một cuộc điều_tra hình_sự " toàn_diện , cật_lực và khách_quan " về cái chết của cha_mẹ họ . Điều_tra_viên phụ_trách án_mạng , ông David_Hopkinson , chia_sẻ với đài Sky_News rằng " vụ án có nhiều điểm đáng ngờ và chúng_tôi đang xử_lý vụ_việc theo hướng đó " . Theo báo The_Star ( Canada ) , ông Sherman là một trong những người giàu nhất ở Canada . Vợ_chồng tỉ_phú Sherman đã quyên_góp từ_thiện hàng triệu USD cho nhiều tổ_chức thiện_nguyện . Chính bộ_phận công_tác xã_hội của Công_ty Apotex cũng đã vận_chuyển các đợt thuốc_men hỗ_trợ những vùng khủng_hoảng trị_giá nhiều triệu USD . Vợ_chồng tỉ phủ Canada_Barry_Sherman'

In [200]:
sentence_score_list, sentence_list = score_article(text, 0)

#summary = generate_summary(sentence_score_array, sentence_list)

In [201]:
print(sentence_score_list)

[2.1562970129699295, 1.4325857241515272, 1.60128963968078, 1.4539907682641138, 1.048115301643692, 2.502761832981843, 1.9905970245411577, 1.2587576548035004, 2.5290491836424933, 2.6702851689142264, 1.34650480703621, 1.530755494516098, 1.4023946982185336, 1.4503766362112245, 1.331042482816362, 1.2236336130966687, 0.9838743725182086, 0.3037046932576929]


In [202]:
sentence_score_array = np.asarray(sentence_score_list)
np.mean(sentence_score_array)

1.567556450514681

In [203]:
generate_summary(sentence_score_array, sentence_list)

'Cảnh_sát Canada vừa công_bố nguyên_nhân cái chết của vợ_chồng tỉ_phú Barry_Sherman và bà Honey , song kết_luận sơ_bộ của vụ_việc chỉ góp thêm bí_ẩn cho vụ_việc chấn_động này   Một nguồn_tin cảnh_sát trước đó cho CBC News biết một người ngoài gia_đình Sherman đã phát_hiện ra thi_thể vợ_chồng tỉ_phú , song cảnh_sát không tiết_lộ gì nhiều về quá_trình điều_tra   Tờ The_Globe and Mail dẫn một nguồn thạo tin cho biết người phát_hiện thi_thể là một nhân_viên môi_giới địa_ốc , vốn đang thu_xếp bán ngôi nhà của ông_bà Sherman   Bất_chấp sự " kín_tiếng " của cảnh_sát , nhiều hãng tin đã dẫn chính nguồn_tin từ cảnh_sát cho_hay vụ_việc đang được điều_tra theo hướng " giết người rồi tự_tử " , tức một trong hai người đã đoạt mạng người kia trước khi tự kết_liễu bản_thân   Tuy_nhiên , trong thông_cáo phát đi ngày 16-12 , gia_đình Sherman khẳng_định " cha_mẹ chúng_tôi đã sống rất_đỗi nhiệt_thành và luôn hướng về gia_đình và cộng_đồng , điều này hoàn_toàn trái_ngược với các tin_đồn đáng tiếc trên mạn